In [1]:
import pandas as pd
import numpy as np 

In [2]:
import dash
from dash import dcc
from dash import html
import plotly.express as px

In [25]:
urlI = 'https://www.ndbc.noaa.gov/data/realtime2/KIKT.txt'
urlB = 'https://www.ndbc.noaa.gov/data/realtime2/KBQX.txt'
urlM = 'https://www.ndbc.noaa.gov/data/realtime2/KMIS.txt'

kikt = pd.read_csv(urlI, skiprows=[1], sep='\s+')
kapt = pd.read_csv(urlB, skiprows=[1], sep='\s+')
kmis = pd.read_csv(urlM, skiprows=[1], sep='\s+')


In [26]:
def preproc_data(df):
    df = df.rename(columns={"#YY": "YR"})
    df['datetime'] = pd.to_datetime(df.YR.astype(str) + '-' + df.MM.astype(str) + '-' + df.DD.astype(str) \
                         + ' ' + df.hh.astype(str) + ':' + df.mm.astype(str))
    newdf = df[['datetime', 'WDIR', 'WSPD', 'GST', 'ATMP', 'DEWP', 'VIS']].sort_values(by=['datetime'])
    return newdf 

In [27]:
# preprocessing all the dataframes 
kikt = preproc_data(kikt)
kapt = preproc_data(kapt)
kmis = preproc_data(kmis)

units = np.array(['datetime', 'degT', 'm/s', 'm/s', 'degC', 'degC', 'nmi'])  #units for the remaining columns 

In [28]:
column = "WDIR"
df = kikt

In [31]:
# function to interpolate where columns have MM
def interp_MM(df, column):
    dt = df['datetime'].to_numpy().astype(float)
    dt = (dt - np.min(dt))/(np.max(dt)-np.min(dt))

    x = dt[df[column] == 'MM']
    xp = dt[df[column] != 'MM']
    yp = df.loc[df[column]!= 'MM', column].to_numpy().astype(float)

    testing = np.interp(x, xp, yp) 
    
    df.loc[(df[column] == 'MM'), column] =  testing

    return df

In [32]:
for i in kikt.columns[1:]:
    kikt = interp_MM(kikt, i)
    kapt = interp_MM(kapt, i)
    kmis = interp_MM(kmis, i)

In [33]:
kikt

,datetime,WDIR,WSPD,GST,ATMP,DEWP,VIS
1324,2021-12-02 22:35:00,230.0,1.5,6.2,21,16,8.7
1323,2021-12-02 22:55:00,230.0,0,6.2,21,16,8.7
1322,2021-12-02 23:35:00,230.0,0,6.2,21,16,8.7
1321,2021-12-02 23:55:00,230.0,0,6.2,21,16,8.7
1320,2021-12-03 00:15:00,230.0,0,6.2,21,16,8.7
...,...,...,...,...,...,...,...
4,2021-12-21 18:55:00,300.0,3.1,8.8,16,10,8.7
3,2021-12-21 19:15:00,300,4.1,8.8,16,10,8.7
2,2021-12-21 19:35:00,300.0,3.1,6.2,15,10,8.7
1,2021-12-21 19:55:00,300,3.6,9.3,15,10,8.7
